# Pk Emulator

In [3]:
import pyccl as ccl
import numpy as np
import pylab as plt
import itertools

import pymaster as nmt

First we want to set up a grid of h and Omega_m spanning across our priors 

In [4]:
w_arr  = np.linspace(0.1, 1, 10)
h_arr  = np.linspace(0.1,  1, 10)

In [5]:
cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.049, h=0.67, sigma8=0.81, n_s=0.96)

In [6]:
nk = 100 # No. of points in k
k_arr = np.logspace(-4.7,5,nk)

In [7]:
data = []
for i in range(len(w_arr)):
    for j in range(len(h_arr)):
        cosmo = ccl.Cosmology(Omega_c=w_arr[i], Omega_b=0.049, h=h_arr[j], sigma8=0.81, n_s=0.96)
        data.append(ccl.power.linear_matter_power(cosmo, k_arr, 1))
data = np.array(data)

KeyboardInterrupt: 

In [ ]:
data.shape

Take log and remove mean

In [ ]:
log_data = np.log(data)
log_mean_data = np.log(np.mean(data, axis=0)) #avg over all cosmologies
clean_data = log_data - log_mean_data

In [ ]:
clean_data.shape

Now make PCA to remove number of samples

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(np.transpose(clean_data))

In [ ]:
pca.components_

In [ ]:
for i in range(len(log_data)) :   
    plt.plot( log_data[i], 'b-', alpha=0.3)
plt.plot( log_mean_data, 'r-', label='Log mean data')
plt.xlabel('k', fontsize=15)
plt.ylabel('ln Pk', fontsize=15)
plt.legend()
plt.show()

In [ ]:
for i in range(len(clean_data)):
    plt.plot( clean_data[i], 'b-', alpha=0.2)
plt.xlabel('k', fontsize=15)
plt.ylabel('ln Pk', fontsize=15)
plt.legend()
plt.show()

In [ ]:
 principalComponents = np.transpose( principalComponents)

In [ ]:
plt.plot( principalComponents[0], 'g-',label='PCA1')
plt.plot( principalComponents[1], 'r-',label='PCA2')
plt.plot( principalComponents[2], 'b-',label='PCA3')
plt.plot( principalComponents[3], 'y-',label='PCA4')
plt.plot( log_mean_data, 'k-', label='Log mean data')
plt.xlabel('k', fontsize=15)
plt.ylabel('ln Pk', fontsize=15)
plt.legend()
plt.show()

Now we need to match estimates based on this PC to actual spectra 

In [ ]:
def theory_Pk(weigths , evecs, mean):
    return mean + np.sum(evecs*weigths, axis=0)

In [ ]:
weigths1 = np.array([[1],[0],[1],[0]])
prediction1 = theory_Pk(weigths1, principalComponents, log_mean_data)
weigths2 = np.array([[0],[1],[0],[1]])
prediction2 = theory_Pk(weigths2, principalComponents, log_mean_data)

In [ ]:
plt.plot( prediction1, 'r--',label='Theory 1')
plt.plot( prediction2, 'g--',label='Theory 2')
plt.plot( log_mean_data, 'b-',label='Log mean Pk')
plt.xlabel('k', fontsize=15)
plt.ylabel('ln Pk', fontsize=15)
plt.legend()
plt.show()

# Interpolation method

In [9]:
import scipy
import pyccl as ccl
import numpy as np
import pylab as plt
import itertools

import pymaster as nmt

In [10]:
def theory_Pk(k, wc_arr, h_arr):
    output = []
    for i in range(len(wc_arr)):
        row = []
        for j in range(len(h_arr)):
            cosmo = ccl.Cosmology(Omega_c=wc_arr[i], Omega_b=0.049, h=h_arr[j], sigma8=0.81, n_s=0.96)
            row.append(np.log(ccl.power.linear_matter_power(cosmo, k, 1)))
        output.append(np.array(row))
    return np.array(output)

In [11]:
wc_arr  = np.linspace(0.1, 1, 4)
h_arr  = np.linspace(0.1,  1, 5)
Pk_arr  = theory_Pk(1, wc_arr, h_arr)

In [12]:
print(wc_arr.size)
print(h_arr.size)
print(Pk_arr.shape)

4
5
(4, 5)


In [13]:
scipy.interpolate.interp2d(h_arr, wc_arr, Pk_arr, kind='cubic')

In [14]:
nk = 100 # No. of points in k
k_arr = np.logspace(-4.7,5,nk)
k_arr

array([1.99526231e-05, 2.50022906e-05, 3.13299424e-05, 3.92590145e-05,
       4.91947992e-05, 6.16451608e-05, 7.72464959e-05, 9.67962619e-05,
       1.21293739e-04, 1.51991108e-04, 1.90457456e-04, 2.38658979e-04,
       2.99059481e-04, 3.74746317e-04, 4.69588196e-04, 5.88432931e-04,
       7.37355235e-04, 9.23967224e-04, 1.15780751e-03, 1.45082878e-03,
       1.81800871e-03, 2.27811560e-03, 2.85466766e-03, 3.57713519e-03,
       4.48244688e-03, 5.61687746e-03, 7.03841301e-03, 8.81971488e-03,
       1.10518338e-02, 1.38488637e-02, 1.73537741e-02, 2.17457173e-02,
       2.72491861e-02, 3.41454887e-02, 4.27871275e-02, 5.36158171e-02,
       6.71850628e-02, 8.41884522e-02, 1.05495109e-01, 1.32194115e-01,
       1.65650183e-01, 2.07573409e-01, 2.60106686e-01, 3.25935236e-01,
       4.08423865e-01, 5.11788955e-01, 6.41313980e-01, 8.03619571e-01,
       1.00700193e+00, 1.26185688e+00, 1.58121126e+00, 1.98138877e+00,
       2.48284435e+00, 3.11120976e+00, 3.89860370e+00, 4.88527357e+00,
      